In [88]:
!pip install pyspark

In [141]:
import pyspark
import pandas as pd
from pyspark.sql.types import *

In [142]:
import numpy as np
from pyspark.sql import SparkSession


In [143]:
import sys
import math

In [144]:
spark=SparkSession.builder.appName('firstapp').getOrCreate()

In [231]:
#load the dataset and add an index for each rows
data = spark.read.csv('./test.csv', header=True, inferSchema=True)
data = data.drop('Name')
df = data.toPandas()
df_index = pd.DataFrame({'index': range(0, len(df))})
result_df = pd.concat([df, df_index], axis=1)
data = spark.createDataFrame(result_df)
# asf = spark.createDataFrame(ee)
# asf.show()

In [265]:
#Select randomly five initials centroids
centers = data.toPandas().sample(n=5)
for i, ce in centers.iterrows():
    print(ce['Age'])
print(centers)
data.toPandas()[data.toPandas()['index'] == 8].iloc[0]
# npCenters = spark.createDataFrame(interDf)
# centers = npCenters

75
200
400
450
675
    Age    X   Y    Z  index
2    75   30   6   15      2
7   200  105  16   45      7
15  400  225  32   93     15
17  450  255  36  105     17
26  675  390  54  159     26


Age      225
X        120
Y         18
Z         51
index      8
Name: 8, dtype: int64

In [253]:
#Map operation
cols = [item for item in data.columns if(item != 'index')]
def euclideanDistance(x, y):
    dist = 0
    for dim in cols:
        dist = dist + (float(x[dim]) - float(y[dim]))**2
    dist = math.sqrt(dist)
    return dist
def map(x, cent):
    minDist = sys.float_info.max
    index = -1
    for i, center in cent.iterrows():
        distCenter = euclideanDistance(x, center)
        if(minDist > distCenter):
             minDist = distCenter
             index = int(center['index'])
        y = x.asDict()
        # del y['index']
        y['num'] = 1
        toSave = Row(**y)         
    return (index, toSave)
# rdd = data.rdd.map(lambda x: map(x)).groupByKey()
# e = rdd.toDF()
# for f in e.collect():
#     print(f)


In [254]:
def reduce(a, b):
    c = {}
    for column in cols:
        c[column] = float(a[column]) + float(b[column])
    c['num'] = float(a['num']) + float(b['num'])
    toSave = Row(**c)
    return toSave
def computeMean(y):
    g = y[1].asDict()
    for col in cols:
        g[col] = g[col]/g['num']
    g['index'] = int(y[0])
    return Row(**g)

In [268]:
centers = data.toPandas().sample(n=5)
for i in range(0, 5):
    mapResult = data.rdd.map(lambda x: map(x, centers))
    for value in mapResult.collect():
        print(value)
    error = mapResult.map(lambda x: (1, euclideanDistance(x[1], centers[centers['index'] == int(x[0])].iloc[0]))).reduceByKey(lambda a,b: a+b)
    for p in error.collect():
        print(p)
    reduceResult = mapResult.reduceByKey(lambda a,b: reduce(a, b)).map(lambda q: computeMean(q))
    e = reduceResult.toDF()
    for f in e.collect():
        print(f)
    print()
    print()
    centers = e.toPandas()

(11, Row(Age=25, X=5, Y=2, Z=3, index=0, num=1))
(11, Row(Age=50, X=15, Y=4, Z=9, index=1, num=1))
(11, Row(Age=75, X=30, Y=6, Z=15, index=2, num=1))
(11, Row(Age=100, X=45, Y=8, Z=21, index=3, num=1))
(11, Row(Age=125, X=60, Y=10, Z=27, index=4, num=1))
(11, Row(Age=150, X=75, Y=12, Z=33, index=5, num=1))
(11, Row(Age=175, X=90, Y=14, Z=39, index=6, num=1))
(11, Row(Age=200, X=105, Y=16, Z=45, index=7, num=1))
(11, Row(Age=225, X=120, Y=18, Z=51, index=8, num=1))
(11, Row(Age=250, X=135, Y=20, Z=57, index=9, num=1))
(11, Row(Age=275, X=150, Y=22, Z=63, index=10, num=1))
(11, Row(Age=300, X=165, Y=24, Z=69, index=11, num=1))
(11, Row(Age=325, X=180, Y=26, Z=75, index=12, num=1))
(13, Row(Age=350, X=195, Y=28, Z=81, index=13, num=1))
(13, Row(Age=375, X=210, Y=30, Z=87, index=14, num=1))
(17, Row(Age=400, X=225, Y=32, Z=93, index=15, num=1))
(17, Row(Age=425, X=240, Y=34, Z=99, index=16, num=1))
(17, Row(Age=450, X=255, Y=36, Z=105, index=17, num=1))
(17, Row(Age=475, X=270, Y=38, Z=111

(1, 2712.3056420083303)


Row(Age=537.5, X=307.5, Y=43.0, Z=126.0, num=2.0, index=21)
Row(Age=650.0, X=375.0, Y=52.0, Z=153.0, num=7.0, index=23)
Row(Age=175.0, X=90.38461538461539, Y=14.0, Z=39.0, num=13.0, index=11)
Row(Age=362.5, X=202.5, Y=29.0, Z=84.0, num=2.0, index=13)
Row(Age=450.0, X=255.0, Y=36.0, Z=105.0, num=5.0, index=17)


(11, Row(Age=25, X=5, Y=2, Z=3, index=0, num=1))
(11, Row(Age=50, X=15, Y=4, Z=9, index=1, num=1))
(11, Row(Age=75, X=30, Y=6, Z=15, index=2, num=1))
(11, Row(Age=100, X=45, Y=8, Z=21, index=3, num=1))
(11, Row(Age=125, X=60, Y=10, Z=27, index=4, num=1))
(11, Row(Age=150, X=75, Y=12, Z=33, index=5, num=1))
(11, Row(Age=175, X=90, Y=14, Z=39, index=6, num=1))
(11, Row(Age=200, X=105, Y=16, Z=45, index=7, num=1))
(11, Row(Age=225, X=120, Y=18, Z=51, index=8, num=1))
(11, Row(Age=250, X=135, Y=20, Z=57, index=9, num=1))
(13, Row(Age=275, X=150, Y=22, Z=63, index=10, num=1))
(13, Row(Age=300, X=165, Y=24, Z=69, index=11, num=1))
(13, Row(Age=325, X=180, Y=26, Z=75, index=12, num=1))

(1, 1549.8125528015667)
Row(Age=537.5, X=307.5, Y=43.0, Z=126.0, num=4.0, index=21)
Row(Age=662.5, X=382.5, Y=53.0, Z=156.0, num=6.0, index=23)
Row(Age=137.5, X=68.0, Y=11.0, Z=30.0, num=10.0, index=11)
Row(Age=337.5, X=187.5, Y=27.0, Z=78.0, num=6.0, index=13)
Row(Age=450.0, X=255.0, Y=36.0, Z=105.0, num=3.0, index=17)




(11, Row(Age=25, X=5, Y=2, Z=3, index=0, num=1))
(11, Row(Age=50, X=15, Y=4, Z=9, index=1, num=1))
(11, Row(Age=75, X=30, Y=6, Z=15, index=2, num=1))
(11, Row(Age=100, X=45, Y=8, Z=21, index=3, num=1))
(11, Row(Age=125, X=60, Y=10, Z=27, index=4, num=1))
(11, Row(Age=150, X=75, Y=12, Z=33, index=5, num=1))
(11, Row(Age=175, X=90, Y=14, Z=39, index=6, num=1))
(11, Row(Age=200, X=105, Y=16, Z=45, index=7, num=1))
(11, Row(Age=225, X=120, Y=18, Z=51, index=8, num=1))
(13, Row(Age=250, X=135, Y=20, Z=57, index=9, num=1))
(13, Row(Age=275, X=150, Y=22, Z=63, index=10, num=1))
(13, Row(Age=300, X=165, Y=24, Z=69, index=11, num=1))
(13, Row(Age=325, X=180, Y=26, Z=75, index=12, num=1))
(13, Row(Age=350, X=195, Y=28, Z=81, index=13, num=1))
(13, Row(Age=375, X=210, Y=30, Z=87, index=14, num=1))
(17, Row(Age=400, X=225, Y=32, Z=93, index=15, num=1))
(17, Row(Age=425, X=240, Y=34, Z=99, index=16, num=1))
(17, Row(Age=450, X=255, Y=36, Z=105, index=17, num=1))
(17, Row(Age=475, X=270, Y=38, Z=111

In [198]:
for a in data.rdd.map(lambda x: map(x, centers)).collect():
    print(a)


(0, Row(Age=25, X=5, Y=2, Z=3, index=0, num=1))
(0, Row(Age=50, X=15, Y=4, Z=9, index=1, num=1))
(0, Row(Age=75, X=30, Y=6, Z=15, index=2, num=1))
(0, Row(Age=100, X=45, Y=8, Z=21, index=3, num=1))
(0, Row(Age=125, X=60, Y=10, Z=27, index=4, num=1))
(0, Row(Age=150, X=75, Y=12, Z=33, index=5, num=1))
(0, Row(Age=175, X=90, Y=14, Z=39, index=6, num=1))
(13, Row(Age=200, X=105, Y=16, Z=45, index=7, num=1))
(13, Row(Age=225, X=120, Y=18, Z=51, index=8, num=1))
(13, Row(Age=250, X=135, Y=20, Z=57, index=9, num=1))
(13, Row(Age=275, X=150, Y=22, Z=63, index=10, num=1))
(13, Row(Age=300, X=165, Y=24, Z=69, index=11, num=1))
(13, Row(Age=325, X=180, Y=26, Z=75, index=12, num=1))
(13, Row(Age=350, X=195, Y=28, Z=81, index=13, num=1))
(13, Row(Age=375, X=210, Y=30, Z=87, index=14, num=1))
(13, Row(Age=400, X=225, Y=32, Z=93, index=15, num=1))
(19, Row(Age=425, X=240, Y=34, Z=99, index=16, num=1))
(19, Row(Age=450, X=255, Y=36, Z=105, index=17, num=1))
(19, Row(Age=475, X=270, Y=38, Z=111, index